In [1]:
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from imblearn.combine import SMOTEENN

**Reading csv**

In [2]:
df=pd.read_csv("/content/tel_churn.csv")
df.head()

,Unnamed: 0,SeniorCitizen,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,0,29.85,29.85,0,1,0,0,1,1,...,0,0,1,0,1,0,0,0,0,0
1,1,0,56.95,1889.50,0,0,1,1,0,1,...,0,0,0,1,0,0,1,0,0,0
2,2,0,53.85,108.15,1,0,1,1,0,1,...,0,0,0,1,1,0,0,0,0,0
3,3,0,42.30,1840.75,0,0,1,1,0,1,...,1,0,0,0,0,0,0,1,0,0
4,4,0,70.70,151.65,1,1,0,1,0,1,...,0,0,1,0,1,0,0,0,0,0


In [3]:
df=df.drop('Unnamed: 0',axis=1)

In [7]:
x=df.drop('Churn',axis=1)
y=df['Churn']

Train Test Split

In [8]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

**Decision Tree Classifie**

In [9]:
model_dt=DecisionTreeClassifier(criterion = "gini",random_state = 100,max_depth=6, min_samples_leaf=8)

In [10]:
model_dt.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [11]:
y_pred=model_dt.predict(x_test)
y_pred

array([0, 0, 0, ..., 1, 0, 0])

In [12]:
model_dt.score(x_test,y_test)

0.7825159914712153

In [13]:
print(classification_report(y_test, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.79      0.94      0.86       996
           1       0.73      0.41      0.52       411

    accuracy                           0.78      1407
   macro avg       0.76      0.67      0.69      1407
weighted avg       0.77      0.78      0.76      1407



As you can see that the accuracy is quite low, and as it's an imbalanced dataset, we shouldn't consider Accuracy as our metrics to measure the model, as Accuracy is cursed in imbalanced datasets.

Hence, we need to check recall, precision & f1 score for the minority class, and it's quite evident that the precision, recall & f1 score is too low for Class 1, i.e. churned customers.

Hence, moving ahead to call SMOTEENN (UpSampling + ENN)

In [15]:
sm = SMOTEENN()
X_resampled, y_resampled = sm.fit_resample(x,y)

In [16]:
xr_train,xr_test,yr_train,yr_test=train_test_split(X_resampled, y_resampled,test_size=0.2)

In [17]:
model_dt_smote=DecisionTreeClassifier(criterion = "gini",random_state = 100,max_depth=6, min_samples_leaf=8)

In [18]:
model_dt_smote.fit(xr_train,yr_train)
yr_predict = model_dt_smote.predict(xr_test)
model_score_r = model_dt_smote.score(xr_test, yr_test)
print(model_score_r)
print(metrics.classification_report(yr_test, yr_predict))

0.9511373209772536
              precision    recall  f1-score   support

           0       0.97      0.92      0.94       530
           1       0.94      0.98      0.96       657

    accuracy                           0.95      1187
   macro avg       0.95      0.95      0.95      1187
weighted avg       0.95      0.95      0.95      1187



In [19]:
print(metrics.confusion_matrix(yr_test, yr_predict))

[[488  42]
 [ 16 641]]


Now we can see quite better results, i.e. Accuracy: 95 %, and a very good recall, precision & f1 score for minority class.

Let's try with some other classifier.

**Random Forest Classifier**

In [22]:
from sklearn.ensemble import RandomForestClassifier

In [20]:
xr_train_rf,xr_test_rf,yr_train_rf,yr_test_rf=train_test_split(X_resampled, y_resampled,test_size=0.2)

In [23]:
model_rf_smote=RandomForestClassifier(n_estimators=100, criterion='gini', random_state = 100,max_depth=6, min_samples_leaf=8)

In [24]:
model_rf_smote.fit(xr_train_rf,yr_train_rf)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [25]:
yr_predict1 = model_rf_smote.predict(xr_test_rf)

In [26]:
model_score_r1 = model_rf_smote.score(xr_test_rf, yr_test_rf)

In [27]:
print(model_score_r1)
print(metrics.classification_report(yr_test_rf, yr_predict1))

0.9199663016006739
              precision    recall  f1-score   support

           0       0.95      0.87      0.91       542
           1       0.90      0.96      0.93       645

    accuracy                           0.92      1187
   macro avg       0.92      0.92      0.92      1187
weighted avg       0.92      0.92      0.92      1187



Performing PCA

In [28]:
# Applying PCA
from sklearn.decomposition import PCA
pca = PCA(0.9)
xr_train_pca = pca.fit_transform(xr_train_rf)
xr_test_pca = pca.transform(xr_test_rf)
explained_variance = pca.explained_variance_ratio_

In [29]:
model=RandomForestClassifier(n_estimators=100, criterion='gini', random_state = 100,max_depth=6, min_samples_leaf=8)

In [31]:
model.fit(xr_train_pca,yr_train_rf)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [32]:
yr_predict_pca = model.predict(xr_test_pca)

In [33]:
model_score_r_pca = model.score(xr_test_pca, yr_test_rf)

In [35]:
print(model_score_r_pca)
print(metrics.classification_report(yr_test_rf, yr_predict_pca))

0.7320977253580455
              precision    recall  f1-score   support

           0       0.72      0.68      0.70       542
           1       0.74      0.78      0.76       645

    accuracy                           0.73      1187
   macro avg       0.73      0.73      0.73      1187
weighted avg       0.73      0.73      0.73      1187



With PCA, we couldn't see any better results, hence let's finalise the model which was created by RF Classifier, and save the model so that we can use it in a later stage :)

**Pickling the model**

In [36]:
import pickle

In [37]:
filename = 'model.sav'

In [38]:
pickle.dump(model_rf_smote, open(filename, 'wb'))

In [39]:
load_model = pickle.load(open(filename, 'rb'))

In [40]:
model_score_r1 = load_model.score(xr_test, yr_test)

In [41]:
model_score_r1

0.945240101095198

Our final model i.e. DT Classifier with SMOTEENN, is now ready and dumped in model.sav, which we will use and prepare API's so that we can access our model from UI.